In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

from PhaseFunctionedNetwork import PhaseFunctionedNetwork
from train_utils import train_pfnn

# set seeds for reproduceability
torch.manual_seed(42)
np.random.seed(42)
rng = np.random.RandomState(42)

In [2]:
# Process Data 

# Idle Data
X_idle = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Input_Idle_Static.txt'))
Y_idle = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Output_Idle_Static.txt'))
P_idle = np.linspace(0, 2*np.pi, num=72)
delta_idle =  P_idle[1:] - P_idle[:-1]
delta_idle[delta_idle < 0] = (1.0 - P_idle[:-1] + P_idle[1:])[delta_idle < 0]
delta_idle = np.append(delta_idle, 0) #TODO IF BREAKS append 2pi
print(f"Idle data shape: {X_idle.shape}, {Y_idle.shape}, {P_idle.shape}, {delta_idle.shape}")

# Jump Flat
X_jump_horizflat = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Input_Jump_HorizFlat.txt'))
Y_jump_horizflat = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Output_Jump_HorizFlat.txt'))
P_jump_horizflat = np.linspace(0, 2*np.pi, num=50)
delta_jump_horizflat =  P_jump_horizflat[1:] - P_jump_horizflat[:-1]
delta_jump_horizflat[delta_jump_horizflat < 0] = (1.0 - P_jump_horizflat[:-1] + P_jump_horizflat[1:])[delta_jump_horizflat < 0]
delta_jump_horizflat = np.append(delta_jump_horizflat, 0) #TODO IF BREAKS append 2pi
print(f"Horizflat data shape: {X_jump_horizflat.shape}, {Y_jump_horizflat.shape}, {P_jump_horizflat.shape}, {delta_jump_horizflat.shape}")

# Jump Up
X_jump_horizup = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Input_Jump_HorizUp.txt'))
Y_jump_horizup = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Output_Jump_HorizUp.txt'))
P_jump_horizup = np.linspace(0, 2*np.pi, num=48)
delta_jump_horizup =  P_jump_horizup[1:] - P_jump_horizup[:-1]
delta_jump_horizup[delta_jump_horizup < 0] = (1.0 - P_jump_horizup[:-1] + P_jump_horizup[1:])[delta_jump_horizup < 0]
delta_jump_horizup = np.append(delta_jump_horizup, 0) #TODO IF BREAKS append 2pi
print(f"Horizup data shape: {X_jump_horizup.shape}, {Y_jump_horizup.shape}, {P_jump_horizup.shape}, {delta_jump_horizup.shape}")

Idle data shape: (72, 1848), (72, 1767), (72,), (72,)
Horizflat data shape: (50, 1848), (50, 1767), (50,), (50,)
Horizup data shape: (48, 1848), (48, 1767), (48,), (48,)


In [7]:
Y_jump_horizflat.shape

(50, 1767)

In [22]:
# X root translation
print(f"X Root translation: {Y_jump_horizflat[:, -3]}")
print(f"Y Root rotation: {Y_jump_horizflat[:, -2]}")
print(f"Z Root translation: {Y_jump_horizflat[:, -1]}")

X Root translation: [ 1.00e-05  2.00e-05  3.00e-05  4.00e-05  3.00e-05 -9.00e-05 -2.60e-04
 -4.30e-04 -5.80e-04 -7.50e-04 -7.50e-04 -6.20e-04 -3.40e-04 -3.00e-05
  0.00e+00  7.00e-05  4.70e-04  1.31e-03  1.63e-03  1.41e-03  1.38e-03
  1.43e-03  1.39e-03  1.22e-03  1.20e-03  1.36e-03  1.42e-03  1.34e-03
  1.09e-03  7.40e-04  3.70e-04 -2.36e-03 -4.18e-03 -4.49e-03 -2.79e-03
 -9.00e-05 -1.20e-04  1.00e-05  1.50e-04  2.20e-04  2.60e-04  2.70e-04
  2.50e-04  2.10e-04  1.70e-04  1.30e-04  7.00e-05  2.00e-05  0.00e+00
 -2.00e-05]
Y Root rotation: [ 0.       0.       0.       0.       0.       0.       0.       0.
  0.       0.       0.       0.       0.       0.       0.       0.
  0.       0.       0.       0.       0.      -0.07133 -0.11014 -0.13706
 -0.14671 -0.15066 -0.15323 -0.16432 -0.15066 -0.11014 -0.04423  0.06256
  0.11535  0.11014  0.11014  0.10835  0.10653  0.10087  0.09691  0.08847
  0.08156  0.07133  0.06256  0.04846  0.03426  0.01978  0.       0.
  0.       0.     ]
Z Root tran

In [23]:
print(f"X Root translation: {Y_jump_horizup[:, -3]}")
print(f"Y Root rotation: {Y_jump_horizup[:, -2]}")
print(f"Z Root translation: {Y_jump_horizup[:, -1]}")

X Root translation: [ 1.00e-05  2.00e-05  3.00e-05  4.00e-05  3.00e-05 -9.00e-05 -2.60e-04
 -4.30e-04 -5.80e-04 -7.50e-04 -7.50e-04 -6.20e-04 -3.40e-04 -3.00e-05
  0.00e+00  7.00e-05  4.70e-04  1.31e-03  1.63e-03  1.46e-03  1.44e-03
  1.38e-03  1.39e-03  1.55e-03  1.46e-03  1.22e-03  7.20e-04  2.80e-04
  2.30e-04  1.40e-04 -2.50e-04 -8.70e-04 -1.24e-03 -1.68e-03 -1.83e-03
 -1.78e-03 -1.53e-03 -1.05e-03 -4.70e-04 -1.00e-05  6.00e-05  1.20e-04
  1.70e-04  1.90e-04  1.80e-04  1.30e-04  7.00e-05  2.00e-05]
Y Root rotation: [ 0.       0.       0.       0.       0.       0.       0.       0.
  0.       0.       0.       0.       0.       0.       0.       0.
  0.       0.       0.       0.      -0.01978 -0.05234 -0.09279 -0.14804
 -0.16313 -0.15195 -0.13562 -0.10279 -0.09065 -0.08393 -0.05935  0.
  0.02798  0.02798  0.03426  0.04423  0.05595  0.07402  0.09279  0.11191
  0.12195  0.12667  0.12195  0.10835  0.08623  0.05595  0.01978  0.     ]
Z Root translation: [ 1.090e-03  9.500e-04  1.030e-

In [6]:
# WEIGH THE JOINT (in particular all the hand and foot joints need to be weighted less)
JOINT_NUM = 144

joint_weights = np.array([
    1,                                                  # HIPS
    1, 1, 1,                                            # LEFT leg
    1e-10, 1e-10, 1e-10, 1,                             # LEFT foot thumb
    1,                                                  # LEFT foot toe base
    1e-10, 1e-10, 1e-10, 1,                             # LEFT foot index
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # LEFT foot middle
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # LEFT foot ring
    1, 1, 1,                                            # RIGHT leg
    1e-10, 1e-10, 1e-10, 1,                             # RIGHT foot thumb
    1,                                                  # RIGHT foot toe base
    1e-10, 1e-10, 1e-10, 1,                             # RIGHT foot index
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # RIGHT foot middle
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # RIGHT foot ring
    1, 1, 1, 1,                                         # SPINE
    1, 1,                                               # LEFT shoulder
    1, 1,                                               # LEFT arm
    1,                                                  # LEFT hand
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # LEFT hand index
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # LEFT hand middle
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # LEFT hand ring
    1, 1e-10, 1,                                        # LEFT wing feathers large
    1, 1e-10, 1,                                        # LEFT wing feathers medium
    1,                                                  # LEFT wing feathers small
    1, 1, 1, 1, 1, 1, 1,                                # NECK
    1,                                                  # HEAD
    1e-10, 1e-10,                                       # JAW
    1e-10, 1e-10, 1e-10, 1e-10, 1e-10, 1e-10, 1e-10,    # TONUGE
    1e-10, 1e-10, 1e-10,                                # LEFT eye
    1e-10, 1e-10, 1e-10,                                # RIGHT eye
    1, 1,                                               # RIGHT shoulder
    1, 1,                                               # RIGHT arm
    1,                                                  # RIGHT hand
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # RIGHT hand index
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # RIGHT hand middle
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # RIGHT hand ring
    1, 1e-10, 1,                                        # RIGHT wing feathers large
    1, 1e-10, 1,                                        # RIGHT wing feathers medium
    1,                                                  # RIGHT wing feathers small
    1, 1, 1, 1, 1, 1, 1, 1, 1,                          # TAIL
    1e-10, 1e-10, 1,                                    # LEFT tail feather
    1e-10, 1e-10, 1,                                    # MIDDLE tail feather         
    1e-10, 1e-10, 1                                     # RIGHT tail feather 
])

# repeat weights for each joint to represent X, Y and Z
joint_weights = joint_weights.repeat(3)

In [3]:
def preprocess_X(X_arr):
    Xmean, Xstd = X_arr.mean(axis=0), X_arr.std(axis=0)

    # lists to keep track of indices for TRAJECTORY
    X_traj_pos_indices = []
    X_traj_dir_indices = []
    X_traj_vel_indices = []
    X_traj_speed_indices = []
    X_traj_style_indices = []

    # number of eleements for each trajectory point
    w = 10
    for i in range(0, 119, w):
        X_traj_pos_indices = np.append(X_traj_pos_indices, range(i,i+3)).astype(int)
        X_traj_dir_indices = np.append(X_traj_dir_indices, range(i+3,i+5)).astype(int)
        X_traj_vel_indices = np.append(X_traj_vel_indices, range(i+5,i+7)).astype(int)
        X_traj_speed_indices = np.append(X_traj_speed_indices, i+7).astype(int)
        X_traj_style_indices = np.append(X_traj_style_indices, range(i+8,i+10)).astype(int)

    # lists to keep track of indices for JOINTS
    X_joint_pos_indices = []
    X_joint_for_indices = []
    X_joint_up_indices = []
    X_joint_vel_indices = []

    # num of elements for each joint
    w = 12
    for i in range(120, 1847, w):
        X_joint_pos_indices = np.append(X_joint_pos_indices, range(i,i+3)).astype(int)
        X_joint_for_indices = np.append(X_joint_for_indices, range(i+3,i+6)).astype(int)
        X_joint_up_indices = np.append(X_joint_up_indices, range(i+6,i+9)).astype(int)
        X_joint_vel_indices = np.append(X_joint_vel_indices, range(i+9,i+12)).astype(int)

    # INPUT Trajectory data
    Xstd[X_traj_pos_indices] = Xstd[X_traj_pos_indices].mean()
    Xstd[X_traj_dir_indices] = Xstd[X_traj_dir_indices].mean()
    Xstd[X_traj_vel_indices] = Xstd[X_traj_vel_indices].mean()
    Xstd[X_traj_speed_indices] = Xstd[X_traj_speed_indices].mean()
    Xstd[X_traj_style_indices] = Xstd[X_traj_style_indices].mean()

    # INPUT Joint data --> This is where we weight the joints
    # Xstd[X_joint_pos_indices] = Xstd[X_joint_pos_indices].mean() / (joint_weights * 0.1)
    # Xstd[X_joint_for_indices] = Xstd[X_joint_for_indices].mean() / (joint_weights * 0.1)
    # Xstd[X_joint_up_indices] = Xstd[X_joint_up_indices].mean() / (joint_weights * 0.1)
    # Xstd[X_joint_vel_indices] = Xstd[X_joint_vel_indices].mean() / (joint_weights * 0.1)
    Xstd[X_joint_pos_indices] = Xstd[X_joint_pos_indices].mean() 
    Xstd[X_joint_for_indices] = Xstd[X_joint_for_indices].mean()
    Xstd[X_joint_up_indices] = Xstd[X_joint_up_indices].mean()
    Xstd[X_joint_vel_indices] = Xstd[X_joint_vel_indices].mean()

    return Xmean, Xstd

def preprocess_Y(Y_arr):
    Ymean, Ystd = Y_arr.mean(axis=0), Y_arr.std(axis=0)

    # PREPROCESS OUTPUT Y
    # lists to keep track of indices for TRAJECTORY
    Y_traj_pos_indices = []
    Y_traj_dir_indices = []
    Y_traj_vel_indices = []

    # number of trajectory elements
    w = 6
    for i in range(0, 35, w): #TODO UPDATE THE RANGE
        Y_traj_pos_indices = np.append(Y_traj_pos_indices, range(i,i+2)).astype(int)
        Y_traj_dir_indices = np.append(Y_traj_dir_indices, range(i+2,i+4)).astype(int)
        Y_traj_vel_indices = np.append(Y_traj_vel_indices, range(i+4,i+6)).astype(int)

    # lists to keep track of indices for JOINTS
    Y_joint_pos_indices = []
    Y_joint_for_indices = []
    Y_joint_up_indices = []
    Y_joint_vel_indices = []

    # num of joint elements
    w = 12
    for i in range(36, 1763, w): #TODO UPDATE THE RANGE
        Y_joint_pos_indices = np.append(Y_joint_pos_indices, range(i,i+3)).astype(int)
        Y_joint_for_indices = np.append(Y_joint_for_indices, range(i+3,i+6)).astype(int)
        Y_joint_up_indices = np.append(Y_joint_up_indices, range(i+6,i+9)).astype(int)
        Y_joint_vel_indices = np.append(Y_joint_vel_indices, range(i+9,i+12)).astype(int)

    # OUTPUT Trajectory data
    Ystd[Y_traj_pos_indices] = Ystd[Y_traj_pos_indices].mean()
    Ystd[Y_traj_dir_indices] = Ystd[Y_traj_dir_indices].mean()
    Ystd[Y_traj_vel_indices] = Ystd[Y_traj_vel_indices].mean()

    # OUTPUT Joint data --> This is where we weight the joints
    # Ystd[Y_joint_pos_indices] = Ystd[Y_joint_pos_indices].mean() / (joint_weights * 0.1)
    # Ystd[Y_joint_for_indices] = Ystd[Y_joint_for_indices].mean() / (joint_weights * 0.1)
    # Ystd[Y_joint_up_indices] = Ystd[Y_joint_up_indices].mean() / (joint_weights * 0.1)
    # Ystd[Y_joint_vel_indices] = Ystd[Y_joint_vel_indices].mean() / (joint_weights * 0.1)
    Ystd[Y_joint_pos_indices] = Ystd[Y_joint_pos_indices].mean()
    Ystd[Y_joint_for_indices] = Ystd[Y_joint_for_indices].mean()
    Ystd[Y_joint_up_indices] = Ystd[Y_joint_up_indices].mean() 
    Ystd[Y_joint_vel_indices] = Ystd[Y_joint_vel_indices].mean()
    
    #TODO MAYBE ROOT TRASLATION arr[-3:] --> last three elements

    return Ymean, Ystd

In [4]:
# Concatenate Data into indivudal X, Y, P Arrays
X = np.concatenate((X_jump_horizflat, X_jump_horizup))
Y = np.concatenate((Y_jump_horizflat, Y_jump_horizup))
P = np.concatenate((P_jump_horizflat, P_jump_horizup))
delta_phase = np.concatenate((delta_jump_horizflat, delta_jump_horizup))
Y = np.concatenate([Y, delta_phase [..., np.newaxis]], axis=-1)

# Preprocess Data
Xmean, Xstd = preprocess_X(X)
Ymean, Ystd = preprocess_Y(Y)

WEIGHTS_SAVE_PATH = 'C:/Users/Ana/Desktop/dev/pfnn-dev/unity-pfnn/Assets/Dev/Weights/test/'

# save means and stds
Xmean.astype(np.float32).tofile(WEIGHTS_SAVE_PATH + 'Xmean.bin')
Ymean.astype(np.float32).tofile(WEIGHTS_SAVE_PATH + 'Ymean.bin')
Xstd.astype(np.float32).tofile(WEIGHTS_SAVE_PATH + 'Xstd.bin')
Ystd.astype(np.float32).tofile(WEIGHTS_SAVE_PATH + 'Ystd.bin')

# normalize data NOTE ORIGINAL DID THIS AFTER SAVING THE MEANS AND STD AS DONE HERE
X = (X - Xmean) / Xstd
Y = (Y - Ymean) / Ystd

# load data for PyTorch training

# append phase as additional feature only for training NN
input = torch.tensor(np.concatenate([X, P [..., np.newaxis]], axis=-1))
target = torch.tensor(Y)

print(f"Input shape {input.shape}")
print(f"Target shape {target.shape}")

dataset = TensorDataset(input, target)

Input shape torch.Size([98, 1849])
Target shape torch.Size([98, 1768])


In [6]:
# Define PFNN
model = PhaseFunctionedNetwork(input_shape=input.shape[1], output_shape=target.shape[1])

# Determine device for training 
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(DEVICE)

# Training variables
BATCH_SIZE = 32
EPOCHS = 5
LR = 0.001
OPTIMIZER = torch.optim.AdamW(model.parameters(), lr=LR)
train_dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# Train
model = train_pfnn(model, train_dataloader, optimizer=OPTIMIZER, num_epochs=EPOCHS, device=DEVICE)

# Save
model.precompute_and_save_weights()

100%|██████████| 4/4 [00:00<00:00,  6.30it/s]


Epoch [1/5], Loss: 1.5537628228930378


100%|██████████| 4/4 [00:00<00:00,  8.05it/s]


Epoch [2/5], Loss: 1.1913716582606715


100%|██████████| 4/4 [00:00<00:00,  8.03it/s]


Epoch [3/5], Loss: 1.145558921869398


100%|██████████| 4/4 [00:00<00:00,  8.01it/s]


Epoch [4/5], Loss: 1.075608583632128


100%|██████████| 4/4 [00:00<00:00,  8.00it/s]


Epoch [5/5], Loss: 0.9982292530218386
